In [8]:
import pyreadr
import math
import numpy as np
import pandas as pd
from pymatreader import read_mat
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import matplotlib
import matplotlib.pyplot as plt
import time

In [2]:
def recursionStart(spectra):
    featurePairs = []
    maxima = []
    minima = []
    
    for i in range(1, len(spectra) - 1):
        if spectra[i] > spectra[i-1]:
            if spectra[i] >= spectra[i+1]:
                for j in range(i+1, len(spectra)):
                    if spectra[i] > spectra[j]:
                        maxima.append([i, spectra[i]])
                        break
                    elif spectra[i] < spectra[j]:
                        break
        if spectra[i] < spectra[i-1]:
            if spectra[i] <= spectra[i+1]:
                for j in range(i+1, len(spectra)):
                    if spectra[i] < spectra[j]:
                        minima.append([i, spectra[i]])
                        break
                    elif spectra[i] > spectra[j]:
                        break
    maxima.sort(key = lambda element: element[1], reverse = True)
    minima.sort(key = lambda element: element[1])
    
    globalMaxima = maxima.pop(0)
    featurePairs.append([globalMaxima[0], globalMaxima[1] - minima[0][1]])
    
    recursionStep(0, globalMaxima[0], maxima.copy(), minima.copy(), featurePairs)
    recursionStep(len(spectra) - 1, globalMaxima[0], maxima.copy(), minima.copy(), featurePairs)
    
    return featurePairs

def recursionStep(start, end, maxima, minima, featurePairs):
    factor = 1
    if end < start:
        factor = -1
    currentMaxima = []
    for i in range(len(maxima)):
        position = maxima[i][0]
        if start * factor < position * factor and position * factor < end * factor:
            currentMaxima.append(maxima[i])
    if len(currentMaxima) == 0:
        return
    localMaxima = currentMaxima.pop(0)
    recursionStep(start, localMaxima[0], currentMaxima.copy(), minima.copy(), featurePairs)
    currentMinima = []
    for i in range(len(minima)):
        position = minima[i][0]
        if localMaxima[0] * factor < position * factor and position * factor < end:
            currentMinima.append(minima[i])
    localMinima = currentMinima.pop(0)
    featurePairs.append([localMaxima[0], localMaxima[1] - localMinima[1]])
    recursionStep(localMinima[0], localMaxima[0], currentMaxima.copy(), currentMinima.copy(), featurePairs)
    recursionStep(localMinima[0], end, currentMaxima.copy(), currentMinima.copy(), featurePairs)

In [3]:
def getPersistenceTransformation(data_X, listOfK):
    spectras = []
    for i in range(len(data_X)):
        featurePairs = recursionStart(data_X.iloc[i])
        featurePairs.sort(key = lambda element: element[1], reverse = True)
        spectras.append(featurePairs)
        
    spectrasForEachK = []
    
    for k in listOfK:
        transformationForSpectra = []
        for i in range(0, len(spectras)):
            transformation = [0] * len(data_X.iloc[i])
            featurePairs = spectras[i][0:round(k*len(spectras[i]))]
            for x, fx in featurePairs:
                transformation[x] = fx
            transformationForSpectra.append(transformation)
        spectrasForEachK.append(transformationForSpectra)
    return spectrasForEachK

In [11]:
def cvAccuracy(tmas, X, resp, classifier):
    
    tmas_names = ['TMA_1:', 'TMA_2:', 'TMA_3:', 'TMA_4:', 'TMA_5:', 'TMA_6:', 'TMA_7:', 'TMA_8:']
    
    tmas_res_test   = []
    tmas_vals_test  = []
    tmas_res_train  = []
    tmas_vals_train = []
    #WE fix this params in order to prevent from overfitting
    
    mtry =  round(math.sqrt(p))
    for tma in range(1, len(tmas_names)+1):
    
        X_train = X.iloc[tmas[tmas != tma].index, :]

        X_test = X.iloc[tmas[tmas == tma].index, :]

        y_train = resp.iloc[tmas[tmas != tma].index, :]

        y_test = resp.iloc[tmas[tmas == tma].index, :]
        
        if (classifier == 'logit'):

            np.random.seed(1234)
            
            logreg = LogisticRegression(penalty = None, solver = 'newton-cg',fit_intercept=True, random_state = 1234)

            logreg.fit(X_train, y_train.values.ravel())

            y_pred = logreg.predict_proba(X_test)

            y_pred_1 = np.where(y_pred[:, 1] > 0.5, 1, 0)

            accuracy = balanced_accuracy_score(y_test, y_pred_1)
            
            tmas_vals_test.append(accuracy)

            tmp_tmas = [tmas_names[tma-1], accuracy]

            tmas_res_test.append(tmp_tmas)
            
        if (classifier == 'rf'):
            
            rf = RandomForestClassifier(n_estimators= 500, random_state= 1234, criterion = 'gini',
                                        max_features=mtry)
            
            rf.fit(X_train, y_train.values.ravel())
            
            y_pred_rf = rf.predict(X_test)
            
            accuracy = balanced_accuracy_score(y_test, y_pred_rf)
            
            tmas_vals_test.append(accuracy)

            tmp_tmas = [tmas_names[tma-1], accuracy]

            tmas_res_test.append(tmp_tmas)
        
    return tmas_vals_test, tmas_res_test  

def Average(lst):
    return sum(lst) / len(lst)   

In [5]:
# Loading Data from Matlab
dataMaldi = read_mat('data/L1-8_tic_ad_sq.mat')

X_values = pd.DataFrame(dataMaldi['data_tic'])#Reading the mz-values 
                                #to get pd's as input for the ml
classes = pd.DataFrame(dataMaldi['classes'])#The cancer Typ, result for ml

mz_values = pd.DataFrame(dataMaldi['mzVector'])

TMAs   = pd.Series(dataMaldi['tmas'])

p = X_values.shape[0]

y = pd.DataFrame(np.where(classes == 1, 0, 1))

In [6]:
# We experiment over a grid of levels of peaks extraction. Namely, 0.01, 0.05, 0.1., 0.2, 0.25, 0.4, 0.5
listOfk = [0.01, 0.05, 0.1, 0.2, 0.25, 0.4, 0.5, 1]
#q = time.time()

#q.tic()
ListOfXk = getPersistenceTransformation(data_X = X_values, listOfK = listOfk)
#q_time = q.tocvalue(restart = True)
#print(f'Time for the analysis : {q_time} seconds')
X_k1 = ListOfXk[0]
X_k2 = ListOfXk[1]
X_k3 = ListOfXk[2]
X_k4 = ListOfXk[3]
X_k5 = ListOfXk[4]
X_k6 = ListOfXk[5]
X_k7 = ListOfXk[6]
X_raw = ListOfXk[7]

In [ ]:
q1, q2 = cvAccuracy(tmas =TMAs, resp=y, X = X_values, classifier='logit')
res_k1_1, res_k1_2 = cvAccuracy(tmas =TMAs, resp=y, X = pd.DataFrame(X_k1), classifier = 'logit')
res_k2_1, res_k2_2 = cvAccuracy(tmas =TMAs, resp=y, X = pd.DataFrame(X_k2), classifier = 'logit')
res_k3_1, res_k3_2 = cvAccuracy(tmas =TMAs, resp=y, X = pd.DataFrame(X_k3), classifier = 'logit')
res_k4_1, res_k4_2 = cvAccuracy(tmas =TMAs, resp=y, X = pd.DataFrame(X_k4), classifier = 'logit')
res_k5_1, res_k5_2 = cvAccuracy(tmas =TMAs, resp=y, X = pd.DataFrame(X_k5), classifier = 'logit')
res_k6_1, res_k6_2 = cvAccuracy(tmas =TMAs, resp=y, X = pd.DataFrame(X_k6), classifier = 'logit')
res_k7_1, res_k7_2 = cvAccuracy(tmas =TMAs, resp=y, X = pd.DataFrame(X_k7), classifier = 'logit')

In [14]:
t = TicToc()

t.tic()
rf_raw_1, rf_raw_2 = cvAccuracy(tmas =TMAs, resp=y, X = X_values, classifier='rf')
t_raw = t.tocvalue(restart = True)
rf_k1_1, rf_k1_2 = cvAccuracy(tmas =TMAs, resp=y, X = pd.DataFrame(X_k1), classifier = 'rf')
t1 = t.tocvalue(restart = True)
rf_k2_1, rf_k2_2 = cvAccuracy(tmas =TMAs, resp=y, X = pd.DataFrame(X_k2), classifier = 'rf')
t2 = t.tocvalue(restart = True)
rf_k3_1, rf_k3_2 = cvAccuracy(tmas =TMAs, resp=y, X = pd.DataFrame(X_k3), classifier = 'rf')
t3 = t.tocvalue(restart = True)
rf_k4_1, rf_k4_2 = cvAccuracy(tmas =TMAs, resp=y, X = pd.DataFrame(X_k4), classifier = 'rf')
t4 = t.tocvalue(restart = True)
rf_k5_1, rf_k5_2 = cvAccuracy(tmas =TMAs, resp=y, X = pd.DataFrame(X_k5), classifier = 'rf')
t5 = t.tocvalue(restart = True)
rf_k6_1, rf_k6_2 = cvAccuracy(tmas =TMAs, resp=y, X = pd.DataFrame(X_k6), classifier = 'rf')
t6 = t.tocvalue(restart = True)
rf_k7_1, rf_k7_2 = cvAccuracy(tmas =TMAs, resp=y, X = pd.DataFrame(X_k7), classifier = 'rf')
t7 = t.tocvalue(restart = True)

print('The calculations took the following times:')
print(f'For the raw peaks it took {t_raw} seconds.')
print(f'For 1% of the peaks it took {t1} seconds.')
print(f'For 5% of the peaks it took {t2} seconds.')
print(f'For 10% of the peaks it took {t3} seconds.')
print(f'For 20% of the peaks it took {t4} seconds.')
print(f'For 25% of the peaks it took {t5} seconds.')
print(f'For 40% of the peaks it took {t6} seconds.')
print(f'For 50% of the peaks it took {t7} seconds.')

KeyboardInterrupt: 

In [ ]:
results_as_tables = pd.concat([pd.DataFrame(res_k1_1).describe(), pd.DataFrame(res_k2_1).describe(),
                              pd.DataFrame(res_k3_1).describe(), pd.DataFrame(res_k4_1).describe(),
                              pd.DataFrame(res_k5_1).describe(), pd.DataFrame(res_k6_1).describe(),
                              pd.DataFrame(res_k7_1).describe(), pd.DataFrame(q1).describe()],
                              axis=1)
results_to_latex = round(results_as_tables, 3)

results_to_latex = results_to_latex.set_axis(['k = 0.01', 'k = 0.05', ' k = 0.1', 'k = 0.2', 
                           'k = 0.25', 'k = 0.4', 'k = 0.5', 'raw'], axis=1)

#pd.DataFrame(results_to_latex).style.to_latex('results/logit_results.tex')

In [ ]:
results_to_latex

In [ ]:
matplotlib.style.use('ggplot')

fig = plt.figure(figsize=(4, 4), dpi=200)
results = pd.concat([pd.Series(res_k2_1), pd.Series(res_k3_1), 
                     pd.Series(res_k4_1), pd.Series(res_k5_1),
                     pd.Series(res_k6_1), pd.Series(res_k7_1), pd.Series(q1)], axis=1)

results = pd.DataFrame(results)

box = results.boxplot()
box.set_xticklabels(['5%', '10%','20%', '25%', '40%', '50%', 'raw'])
box.set_xlabel('Percentage of peak extraction', fontsize=8)
box.set_ylabel('Balanced Accuracy', fontsize=8)
box.set_title('Logistic Regression', fontsize=8)
plt.savefig("results/balanced_accuracy_logit.pdf", format="pdf", bbox_inches="tight")

In [ ]:
results_as_tables_rf = pd.concat([pd.DataFrame(rf_k1_1).describe(),
                              pd.DataFrame(rf_k2_1).describe(), pd.DataFrame(rf_k3_1).describe(),
                              pd.DataFrame(rf_k4_1).describe(), pd.DataFrame(rf_k5_1).describe(),
                              pd.DataFrame(rf_k6_1).describe(), pd.DataFrame(rf_k7_1).describe(), 
                              pd.DataFrame(rf_raw_1).describe()],
                              axis=1)

results_rf_to_latex = round(results_as_tables_rf, 3)

results_rf_to_latex = results_rf_to_latex.set_axis(['k = 0.01', 'k = 0.05', ' k = 0.1', 'k = 0.2', 
                           'k = 0.25', 'k = 0.4', 'k = 0.5', 'raw'], axis=1, inplace=False)

#pd.DataFrame(results_rf_to_latex).style.to_latex('results/results_rf_to_latex.tex')

In [ ]:
results_rf_to_latex

In [ ]:
matplotlib.style.use('ggplot')

fig = plt.figure(figsize=(4, 4), dpi=200)
results = pd.concat([pd.Series(rf_k2_1), pd.Series(rf_k3_1), 
                     pd.Series(rf_k4_1), pd.Series(rf_k5_1),
                     pd.Series(rf_k6_1), pd.Series(rf_k7_1), pd.Series(q1)], axis=1)

results = pd.DataFrame(results)

box = results.boxplot()
box.set_xticklabels(['5%', '10%','20%', '25%', '40%', '50%', 'raw'])
box.set_xlabel('Percentage of peak extraction', fontsize=8)
box.set_ylabel('Balanced Accuracy', fontsize=8)
box.set_title('Random Forest', fontsize=8)
plt.savefig("results/balanced_accuracy_rf.pdf", format="pdf", bbox_inches="tight")

In [ ]:
matplotlib.style.use('ggplot')

fig = plt.figure(figsize=(3, 3), dpi=300)

#plt.subplot(1, 2, 1)
s1 = X_values.loc[[22]]
s2 = pd.DataFrame(X_k1).loc[[22]]
s3 = pd.DataFrame(X_k2).loc[[22]]
s4 = pd.DataFrame(X_k5).loc[[22]]

fig = plt.figure()

fig, ax = plt.subplots(4, 1)


ax[0].plot(mz_values, np.transpose(s1))

ax[0].ticklabel_format(axis='y',style='sci',scilimits=(-3, -3), useMathText = True)
ax[0].set_ylabel('Intensity')
ax[0].set_xlabel('m/z values')
ax[0].set_title("Raw Spectrum")

#plt.subplot(2, 2, 2)
ax[1].plot(mz_values, np.transpose(s2))
ax[1].set_ylabel('Persistance')
ax[1].set_xlabel('m/z-values')
ax[1].set_title("K = 1%")

ax[2].plot(mz_values, np.transpose(s3))
ax[2].set_ylabel('Persistance')
ax[2].set_xlabel('m/z values')
ax[2].set_title("K = 5%")

ax[3].plot(mz_values, np.transpose(s4))
ax[3].set_ylabel('Persistance')
ax[3].set_xlabel('m/z-values')
ax[3].set_title("K = 25%")

plt.subplots_adjust(left=0.1,
                    bottom=0.01,
                    right=1.5,
                    top=2,
                    wspace=0.6,
                    hspace=0.7)

plt.savefig("results/persistance.pdf", format="pdf", bbox_inches="tight")


In [ ]:
import matplotlib
matplotlib.style.use('ggplot')
s1 = X_values.loc[[22]]

fig = plt.figure(figsize=(5,5), dpi=500)

plt.subplot(1, 2, 1)
plt.plot(mz_values, np.transpose( X_values.loc[[22]]))
plt.ylabel('Intensity')
plt.xlabel('m/z values')

plt.ticklabel_format(axis='y',style='sci',scilimits=(-3, -3), useMathText = True)

plt.subplot(1, 2, 2)
plt.plot(mz_values[250:260], np.transpose( X_values.loc[[22]])[250:260], marker = 'o')
plt.ylabel('Intensity')
plt.xlabel('m/z values')


plt.subplots_adjust(left=0.1,
                    bottom=0.1,
                    right=1.5,
                    top=0.85,
                    wspace=0.6,
                    hspace=0.4)

plt.ticklabel_format(axis='y',style='sci',scilimits=(-3, -3), useMathText = True)

plt.savefig("results/example_of_spectra.pdf", format="pdf", bbox_inches="tight")